# Carport (Mesa) Example

Creates a carport of 3x3 (facing south), and shows how to sample along x-axis to get an irradiance map of all the modules. Showing the results for one point in time, for all 3 collectors (each collector consist of the 3 panels in that slope).

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [2]:
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'

timestamp = 4020 # Noon, June 17th. 
simulationname = 'OptionB'

# MakeModule Parameters
moduletype='modified_makeModule'
numpanels = 3
x = 1.4  
y = 0.95 # individual size module
xgap = 0.2 # 10 centimeters.
ygap = 0.1 # you can give ygap now
zgap = 0
sensorsy = 210  # this will give 70 sensors per module.

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = False
cellLevelModule = False

# SceneDict Parameters
gcr = 0.33   # We are only doing 1 row so this doesn't matter
albedo = 0.28  #'concrete'     # ground albedo
clearance_height = 3 # m  
nMods = 3  # replicating the unit 3 times
nRows = 1  # only 1 row

azimuth_ang=180 # Facing south
tilt =20 # tilt. 

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above
demo.gendaylit(metdata,4020)  # Use this to simulate only one hour at a time. 
# This allows you to "view" the scene on RVU (see instructions below)
# timestam 4020 : Noon, June 17th.
#demo.genCumSky(demo.epwfile) # Use this for simulate the whole year


# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y, numpanels = numpanels, xgap=xgap, ygap=ygap)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': round(gcr/moduleDict['sceney'],3),'height':clearance_height,'azimuth':azimuth_ang, 'module_type':moduletype, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.
octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object fil|es into a .oct file.


path = C:\Users\sayala\Documents\RadianceScenes\Demo3
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'modified_makeModule')
REWRITING pre-existing module file. 
Module modified_makeModule successfully created
Created OptionB.oct


In [ ]:
# This adds a marker at the coordinate 0,0
name='Mycube'
text='! genbox black cuteBox 0.05 0.05 5.5 | xform -t -0.025 -0.025 0'
customObject = demo.makeCustomObject(name,text)
demo.appendtoScene(scene.radfiles, customObject, '!xform -rz 0')

octfile = demo.makeOct(demo.getfilelist())  

### At this point you should be able to go into a command window (cmd.exe) and check the geometry. (only if you used gendaylit ! gencumsky oversaturates teh sky). Example:

rvu -vf views\front.vp -e .01 OptionB.oct

Notice that some sensors might fall into the gap betwheen the modules (ygap) so you might have to do some cleaning of the results    

# Now run the analysis

In [ ]:
analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance
modWanted = 1
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(clearanceheight=sceneDict['height'], azimuth=sceneDict['azimuth'], 
                                              tilt=sceneDict['tilt'], 
                                              pitch=sceneDict['pitch'], nMods=sceneDict['nMods'], nRows= sceneDict['nRows'], 
                                              sceney=moduleDict['sceney'], scenex=moduleDict['scenex'], offset=moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)

analysis.analysis(octfile, simulationname+"Mod1", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


modWanted = 2
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(clearanceheight=sceneDict['height'], azimuth=sceneDict['azimuth'], 
                                              tilt=sceneDict['tilt'], 
                                              pitch=sceneDict['pitch'], nMods=sceneDict['nMods'], nRows= sceneDict['nRows'], 
                                              sceney=moduleDict['sceney'], scenex=moduleDict['scenex'], offset=moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)

analysis.analysis(octfile, simulationname+"Mod2", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )


modWanted = 3
rowWanted = 1
frontscan, backscan = analysis.moduleAnalysis(clearanceheight=sceneDict['height'], azimuth=sceneDict['azimuth'], 
                                              tilt=sceneDict['tilt'], 
                                              pitch=sceneDict['pitch'], nMods=sceneDict['nMods'], nRows= sceneDict['nRows'], 
                                              sceney=moduleDict['sceney'], scenex=moduleDict['scenex'], offset=moduleDict['moduleoffset'],
                                              modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)

        
analysis.analysis(octfile, simulationname+"Mod3", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )

### Note that you have to clean the results
As some sensors might fall in the 'gap' between the modules (ygap), and also the sensors are giving results for the 3 modules in the slope at the same time.

# Hacking the analysis example
Just choose another cooridnate for the xstart (since this is facing south it's that simple, you sample along the x-axis, y doesn't change.)

In [ ]:
# HACK Frontscan and Backscan
frontscan['xstart']=-1.2

analysis.analysis(octfile, simulationname+"Mod3_point2", frontscan, backscan)  # compare the back vs front irradiance  
print('Annual bifacial ratio average:  %0.3f' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
